## Step 2: Scouting Research Candidates

Decision whether or not to pursue a research item

In [10]:
import sys
sys.path.append('../')

import pandas as pd
from datetime import datetime

import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

from components.agents.scout_agent import scout_batch

from data.content_saver import ContentSaver
content_saver = ContentSaver(base_path="../data")

2025-12-26 23:46:56,311 - data.content_saver - INFO - Loaded content index with 150 entries


Load Research Items

In [11]:
df = pd.read_csv("../data/research_items.csv")

provider_counts = df['provider'].value_counts()
print("Absolute counts:")
print(provider_counts)
print("\nNormalized (proportions):")
print(df['provider'].value_counts(normalize=True))

Absolute counts:
provider
arxiv        481
openai       359
anthropic    321
exa          195
Name: count, dtype: int64

Normalized (proportions):
provider
arxiv        0.354720
openai       0.264749
anthropic    0.236726
exa          0.143805
Name: proportion, dtype: float64


Select items that have not yet been looked at

In [12]:
pending = df[df.get("scout_decision").isna()] if "scout_decision" in df.columns else df

In [13]:
# Collect scout items
items_to_scout = pending.to_dict("records")
print(f"Number of items to scout: {len(items_to_scout)}")

Number of items to scout: 1356


Run Scouting

In [14]:
decisions = await scout_batch(items_to_scout)

Scout agent triage:   0%|          | 2/1356 [00:20<3:14:13,  8.61s/it]2025-12-26 23:47:38,374 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-12-26 23:47:38,528 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
Scout agent triage:   0%|          | 3/1356 [00:21<1:50:32,  4.90s/it]2025-12-26 23:47:38,842 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-12-26 23:47:38,989 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
Scout agent triage:   0%|          | 5/1356 [00:21<50:49,  2.26s/it]  2025-12-26 23:47:39,280 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-12-26 23:47:39,430 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-12-26 23:47:39,546 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 

Update DataFrame

In [17]:
for i, (idx, row) in enumerate(pending.iterrows()):
    df.loc[idx, "scout_decision"] = "pursue" if decisions[i].pursue else "discard"
    df.loc[idx, "scout_confidence"] = decisions[i].confidence
    df.loc[idx, "scout_reasoning"] = decisions[i].reasoning
    df.loc[idx, "scouted_at"] = datetime.now().isoformat()

In [18]:
pending['scout_decision'].value_counts(dropna=False)

scout_decision
pursue     989
discard    367
Name: count, dtype: int64

Save back to DF

In [19]:
df.to_csv("../data/research_items.csv", index=False)

to test:
1. after new cols ahve been aded, upon running the research, can it still save to it after dedup
3. re-read after some cols already are filled for scouting